In [ ]:
import json
import os
import pandas
import pyarrow
import sys

from dotenv import load_dotenv
from sqlalchemy import create_engine
from os import path

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')

In [ ]:
sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
import create_queries
import prepare_data
%load_ext autoreload
%autoreload 2

In [ ]:
COMPANY_IDENTIFIER = 'RA'
TRANSFER_PACKAGES_START_DATE = '2021-01-01'
TRANSFER_PACKAGES_END_DATE = '2021-09-30'
SALES_TRANSACTIONS_START_DATE = '2021-06-01'
SALES_TRANSACTIONS_END_DATE = '2021-09-30'

In [ ]:
company_incoming_transfer_packages_query = create_queries.create_company_incoming_transfer_packages_query(
    company_identifier=COMPANY_IDENTIFIER,
    start_date=TRANSFER_PACKAGES_START_DATE,
    end_date=TRANSFER_PACKAGES_END_DATE,
)
company_sales_receipts_query = f"""
    select
        metrc_sales_receipts.receipt_id,
        metrc_sales_receipts.receipt_number,
        metrc_sales_receipts.type,
        metrc_sales_receipts.sales_customer_type,
        metrc_sales_receipts.sales_datetime,
        metrc_sales_receipts.total_packages,
        metrc_sales_receipts.total_price
    from
        metrc_sales_receipts
        inner join companies on metrc_sales_receipts.company_id = companies.id
    where
        True
        and companies.identifier = '{COMPANY_IDENTIFIER}'
        and metrc_sales_receipts.sales_datetime >= '{SALES_TRANSACTIONS_START_DATE}'
        and metrc_sales_receipts.sales_datetime <= '{SALES_TRANSACTIONS_END_DATE}'
    order by
        metrc_sales_receipts.sales_datetime desc
"""
company_sales_transactions_query = f"""
    select
        metrc_sales_transactions.receipt_id,
        metrc_sales_transactions.package_id,
        metrc_sales_transactions.package_label,
        metrc_sales_transactions.product_name,
        metrc_sales_transactions.product_category_name,
        metrc_sales_transactions.unit_of_measure,
        metrc_sales_transactions.quantity_sold,
        metrc_sales_transactions.total_price
    from
        metrc_sales_receipts
        inner join companies on metrc_sales_receipts.company_id = companies.id
        left outer join metrc_sales_transactions on metrc_sales_receipts.id = metrc_sales_transactions.receipt_row_id
    where
        True
        and companies.identifier = '{COMPANY_IDENTIFIER}'
        and metrc_sales_receipts.sales_datetime >= '{SALES_TRANSACTIONS_START_DATE}'
        and metrc_sales_receipts.sales_datetime <= '{SALES_TRANSACTIONS_END_DATE}'
    order by
        metrc_sales_receipts.sales_datetime desc
"""

engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))

company_incoming_transfer_packages_dataframe = pandas.read_sql_query(company_incoming_transfer_packages_query, engine)
company_sales_receipts_dataframe = pandas.read_sql_query(company_sales_receipts_query, engine)
company_sales_transactions_dataframe = pandas.read_sql_query(company_sales_transactions_query, engine)

In [ ]:
company_incoming_transfer_packages_dataframe.columns

In [ ]:
company_incoming_transfer_packages_dataframe = company_incoming_transfer_packages_dataframe[[
    'created_date',
    'shipment_transaction_type',
    'package_id',
    'package_label',
    'product_category_name',
    'product_name',
    'shipper_wholesale_price',
    'shipped_quantity',
    'shipped_unit_of_measure',
    'item_unit_weight',
    'item_unit_weight_unit_of_measure_name',
]]
company_incoming_transfer_packages_dataframe

In [ ]:
company_sales_receipts_dataframe

In [ ]:
company_sales_transactions_dataframe

In [ ]:
import time

incoming_packages_xlsx_file_name = f'~/Downloads/take_home_incoming_packages_{round(time.time())}.xlsx'
sales_receipts_xlsx_file_name = f'~/Downloads/take_home_sales_receipts_{round(time.time())}.xlsx'
sales_transactions_xlsx_file_name = f'~/Downloads/take_home_sales_transactions_{round(time.time())}.xlsx'

company_sales_receipts_dataframe['sales_datetime'] = company_sales_receipts_dataframe['sales_datetime'].apply(lambda a: pandas.to_datetime(a).date()) 

company_incoming_transfer_packages_dataframe.to_excel(incoming_packages_xlsx_file_name, index=False)
company_sales_receipts_dataframe.to_excel(sales_receipts_xlsx_file_name, index=False)
company_sales_transactions_dataframe.to_excel(sales_transactions_xlsx_file_name, index=False)